# RNN Autocomplete

In [1]:
import tensorflow as tf
import numpy as np

/Users/hyunwoogu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Define a dictionary

In [2]:
char_arr = ['A', 'B', 'C', 'D', 'E', 'F', 'G',
            'H', 'I', 'J', 'K', 'L', 'M', 'N',
            'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']

num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

### Data

In [3]:
seq_data = ['MOYG', 'MEWG', 'MDVG', 'MQTG', 'XAEH', 'FHRJ', 'LORT', 'LQPW', 'KSVB', 'KJYW']

In [4]:
def make_batch(seq_data):
    input_batch  = []
    target_batch = []

    for seq in seq_data:
        input  = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)

    return input_batch, target_batch

### Options

In [5]:
learning_rate = 0.01
n_hidden      = 128
total_epoch   = 100
n_step        = 3
n_input       = n_class = dic_len

### Variables

In [6]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

### RNN cells

In [7]:
cell1  = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1  = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)

cell2  = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

### Model

In [8]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs   = tf.transpose(outputs, [1, 0, 2])
outputs   = outputs[-1]
model     = tf.matmul(outputs, W) + b

cost      = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Learning

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('Optimization completed!')

Epoch: 0001 cost = 3.253695
Epoch: 0002 cost = 2.384698
Epoch: 0003 cost = 1.473730
Epoch: 0004 cost = 0.782269
Epoch: 0005 cost = 0.329979
Epoch: 0006 cost = 0.151258
Epoch: 0007 cost = 0.041260
Epoch: 0008 cost = 0.013305
Epoch: 0009 cost = 0.003534
Epoch: 0010 cost = 0.021713
Epoch: 0011 cost = 0.009701
Epoch: 0012 cost = 0.000494
Epoch: 0013 cost = 0.000049
Epoch: 0014 cost = 0.002108
Epoch: 0015 cost = 0.000053
Epoch: 0016 cost = 0.002401
Epoch: 0017 cost = 0.000005
Epoch: 0018 cost = 0.878396
Epoch: 0019 cost = 0.000000
Epoch: 0020 cost = 0.000000
Epoch: 0021 cost = 0.063923
Epoch: 0022 cost = 0.131284
Epoch: 0023 cost = 0.002179
Epoch: 0024 cost = 0.148553
Epoch: 0025 cost = 0.000031
Epoch: 0026 cost = 0.000002
Epoch: 0027 cost = 0.000000
Epoch: 0028 cost = 0.000000
Epoch: 0029 cost = 0.000017
Epoch: 0030 cost = 0.000000
Epoch: 0031 cost = 0.000000
Epoch: 0032 cost = 0.000007
Epoch: 0033 cost = 0.000000
Epoch: 0034 cost = 0.000330
Epoch: 0035 cost = 0.000000
Epoch: 0036 cost = 0

### Result

In [10]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [11]:
predict_words = []
for idx, val in enumerate(['MAR']):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in ['MAR']])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['MAR ']
예측값: ['MARG']
정확도: 1.0
